In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.cluster import KMeans
from sklearn import decomposition
from sklearn.manifold import TSNE
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import KFold
import seaborn as sns

Load in data

In [ ]:
profiles_areas = pd.read_csv('~/Desktop/columbia/capstone/fire-regimes/data/profiles-areas.csv')
X = profiles_areas.drop(columns=['_uid_','initialdat','finaldate'])

scaler = StandardScaler()
scaled_data = scaler.fit_transform(X)

X_scaled = X[~(np.abs(scaled_data) > 10).any(axis=1)]
X_scaled = scaler.fit_transform(X)

### An AutoEncoder Class with dynamic latent dimension

In [8]:
class AutoEncoder(torch.nn.Module):
    def __init__(self, layers):
        super(AutoEncoder, self).__init__()
        
        #Encoder
        encoder_layers = []
        input_size = 174
        for layer_size in layers:
            encoder_layers.append(torch.nn.Linear(input_size, layer_size))
            encoder_layers.append(torch.nn.ReLU())
            input_size = layer_size
        self.encoder = torch.nn.Sequential(*encoder_layers)
        
        #Decoder
        decoder_layers = []
        reversed_layers = layers[::-1]
        for layer_size in reversed_layers[1:]:
            decoder_layers.append(torch.nn.Linear(input_size, layer_size))
            decoder_layers.append(torch.nn.ReLU())
            input_size = layer_size
        decoder_layers.append(torch.nn.Linear(input_size, 174))
        self.decoder = torch.nn.Sequential(*decoder_layers)

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [9]:
def train_model(model,train_loader,n_epochs,optimizer,loss_function):

  losses = []

  for _ in range(n_epochs):
    for profile in train_loader:
        
      # Output of Autoencoder
      reconstructed = model(profile)
        
      # Calculating the loss function
      loss = loss_function(reconstructed, profile)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
        
      losses.append(loss.item())
  
  return (model,losses)

In [10]:
def rolling_average(data, window_size):
    return np.convolve(data, np.ones(window_size) / window_size, mode='valid')

def plot_loss(losses,window_size):
    loss_floats = [loss for loss in losses]
    smoothed_losses = rolling_average(loss_floats, window_size)
    plt.plot(smoothed_losses)
    plt.show()

In [ ]:
train_data = torch.from_numpy(X_scaled).float()
train_data = train_data[:, :-1]
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=64, shuffle=True)

In [15]:
latent_dims = [4,6,8,12,16,24,32,48]
results = []

for i,latent_dim in enumerate(latent_dims):
    fold_losses = []
    kf = KFold(n_splits=5, shuffle=True, random_state=1)
    
    for train_idx, val_idx in kf.split(train_data):
        # Split data
        train_subset = torch.utils.data.Subset(train_data, train_idx)
        val_subset = torch.utils.data.Subset(train_data, val_idx)
        train_loader = torch.utils.data.DataLoader(train_subset, batch_size=32, shuffle=True)
        val_loader = torch.utils.data.DataLoader(val_subset, batch_size=32, shuffle=False)

        # Initialize model, optimizer, and loss function for this fold
        model = AutoEncoder(latent_dim=latent_dim)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-8)
        loss_function = torch.nn.MSELoss()

        # Train model and get average training loss for the fold
        avg_train_loss = train_model(model, train_loader=train_loader, n_epochs=2, optimizer=optimizer, loss_function=loss_function)

        model.eval()
        val_losses = []
        with torch.no_grad():
            for profile in val_loader:
                reconstructed = model(profile)
                val_loss = loss_function(reconstructed, profile)
                val_losses.append(val_loss.item())

        avg_val_loss = np.mean(val_losses)
        fold_losses.append(avg_val_loss)

    # Record average validation loss for this latent dimension
    results.append(np.mean(fold_losses))
    print(f"Latent Dim: {latent_dim}, Avg Validation Loss: {results[i]}")

Latent Dim: 4, Avg Validation Loss: 6.233581316412536e-07
Latent Dim: 6, Avg Validation Loss: 5.260538010285384e-07
Latent Dim: 8, Avg Validation Loss: 4.7482631847733987e-07
Latent Dim: 12, Avg Validation Loss: 6.272702753668034e-07
Latent Dim: 16, Avg Validation Loss: 6.781426023215171e-07
Latent Dim: 24, Avg Validation Loss: 5.517599806729095e-07
Latent Dim: 32, Avg Validation Loss: 4.95390734594458e-07
Latent Dim: 48, Avg Validation Loss: 5.867710998081921e-07


In [16]:
model = AutoEncoder(layers=[128,64,16,6])
 
# Validation using MSE Loss function
loss_function = torch.nn.MSELoss()
 
optimizer = torch.optim.Adam(model.parameters(),
                             lr = 0.01,
                             weight_decay = 1e-9)

In [ ]:
model,losses = train_model(model,n_epochs=2,train_loader=train_loader,loss_function=loss_function,optimizer=optimizer)
plot_loss(losses=losses,window_size=20)

In [18]:
torch.save(model.state_dict(), '../assets/AE-6d.pth')

In [19]:
r = model(train_data).detach().numpy()
train = train_data.numpy()
l = model.encoder(train_data).detach().numpy()

In [ ]:
sns.pairplot(pd.DataFrame(l))